In [1]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
OANDA_API_KEY = os.environ["OANDA_API_KEY"]
OANDA_URL = os.environ.get(
    "OANDA_URL",
    "https://api-fxpractice.oanda.com/v3/instruments/{}/candles"
)
OANDA_ACCOUNT_ID = os.environ["OANDA_ACCOUNT_ID"]

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": "Bearer " + OANDA_API_KEY,
    "Content-Type": "application/json"
})

In [6]:
params = {
     
    "count": 10,
    "granularity": "H1",
    "price": "MBA"
    
   
}

In [7]:
url = f"{OANDA_URL}/accounts/{OANDA_ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=params, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()
data

{'instruments': [{'name': 'TRY_JPY',
   'type': 'CURRENCY',
   'displayName': 'TRY/JPY',
   'pipLocation': -2,
   'displayPrecision': 3,
   'tradeUnitsPrecision': 0,
   'minimumTradeSize': '1',
   'maximumTrailingStopDistance': '100.000',
   'minimumTrailingStopDistance': '0.050',
   'maximumPositionSize': '0',
   'maximumOrderUnits': '100000000',
   'marginRate': '0.25',
   'guaranteedStopLossOrderMode': 'DISABLED',
   'tags': [{'type': 'ASSET_CLASS', 'name': 'CURRENCY'},
    {'type': 'BRAIN_ASSET_CLASS', 'name': 'FX'}],
   'financing': {'longRate': '0.2788',
    'shortRate': '-0.402',
    'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
     {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
     {'dayOfWeek': 'FRIDAY', 'daysCharged': 1},
     {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
     {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
  {'name': 'AUD_JPY',
   'type': 'CU

In [11]:
instruments_list = data['instruments']

In [12]:
len(instruments_list)

127

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_interest = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [15]:
instruments_dict = {}
for instrument in instruments_list:
    key = instrument['name']
    instruments_dict[key] = {k: instrument[k] for k in key_interest}

In [16]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [17]:
with open('instruments.json', 'w') as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [18]:
def fetch_candles(pair_name, granularity="M5", count=5000):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = {
        "granularity": granularity,
        "count": count,
        "price": "MBA"
    }
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
    
   

In [19]:
code, data = fetch_candles("EUR_USD", granularity="H1", count=20)
len(data)

20

In [20]:
data

[{'complete': True,
  'volume': 6781,
  'time': '2025-10-20T10:00:00.000000000Z',
  'bid': {'o': '1.16613', 'h': '1.16624', 'l': '1.16504', 'c': '1.16525'},
  'mid': {'o': '1.16620', 'h': '1.16632', 'l': '1.16512', 'c': '1.16534'},
  'ask': {'o': '1.16628', 'h': '1.16639', 'l': '1.16520', 'c': '1.16542'}},
 {'complete': True,
  'volume': 7070,
  'time': '2025-10-20T11:00:00.000000000Z',
  'bid': {'o': '1.16525', 'h': '1.16557', 'l': '1.16436', 'c': '1.16557'},
  'mid': {'o': '1.16534', 'h': '1.16564', 'l': '1.16444', 'c': '1.16564'},
  'ask': {'o': '1.16543', 'h': '1.16572', 'l': '1.16452', 'c': '1.16572'}},
 {'complete': True,
  'volume': 9968,
  'time': '2025-10-20T12:00:00.000000000Z',
  'bid': {'o': '1.16559', 'h': '1.16596', 'l': '1.16494', 'c': '1.16527'},
  'mid': {'o': '1.16566', 'h': '1.16604', 'l': '1.16502', 'c': '1.16535'},
  'ask': {'o': '1.16573', 'h': '1.16613', 'l': '1.16510', 'c': '1.16543'}},
 {'complete': True,
  'volume': 9844,
  'time': '2025-10-20T13:00:00.0000000